# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [33]:
# import libraries
from sqlalchemy.engine import create_engine
import pandas as pd
import nltk
nltk.download(['punkt', 'wordnet'])

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [34]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table("InsertTableNames", engine)
X = df.message
y = df[list(set(df.columns) - set(df[['id', 'message', 'genre',]]))]

In [35]:
df

,id,message,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,direct,1,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,14,Information about the National Palace-,direct,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,15,Storm at sacred heart of jesus,direct,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
7,16,"Please, we need tents and water. We are in Sil...",direct,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,17,"I would like to receive the messages, thank you",direct,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,18,I am in Croix-des-Bouquets. We have health iss...,direct,1,1,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,1


In [36]:
df.isnull().sum().sum()

0

### 2. Write a tokenization function to process your text data

In [37]:
def tokenize(text):
    # tokenize text
    tokens = word_tokenize(text)
    
    # initialize lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [38]:
def display_result(y_test, y_pred):
    y_pred_df = pd.DataFrame(y_pred, columns=y.columns)
    y_test_df = pd.DataFrame(y_test, columns=y.columns)
    
    for column in y_pred_df.columns:
        print(f'The test result for "{column}" column is:', '\n')
        print(classification_report(y_test_df[column], y_pred_df[column]), '\n')

In [39]:
def build_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X.values, y.values)
    
    clf = RandomForestClassifier()

    pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                    ('tfidf', TfidfTransformer()),
                    ('multi', MultiOutputClassifier(clf))])
    
    pipeline.fit(X_train, y_train)
    
    y_pred = pipeline.predict(X_test)
    
    display_result(y_test, y_pred)


In [40]:
build_model(X, y)

The test result for "shelter" column is: 

             precision    recall  f1-score   support

          0       0.93      1.00      0.96      6016
          1       0.87      0.22      0.36       581

avg / total       0.92      0.93      0.91      6597
 

The test result for "search_and_rescue" column is: 

             precision    recall  f1-score   support

          0       0.97      1.00      0.99      6407
          1       0.40      0.01      0.02       190

avg / total       0.96      0.97      0.96      6597
 

The test result for "water" column is: 

             precision    recall  f1-score   support

          0       0.95      1.00      0.97      6183
          1       0.86      0.24      0.38       414

avg / total       0.95      0.95      0.94      6597
 

The test result for "shops" column is: 

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6570
          1       0.00      0.00      0.00        27

avg / tota

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [42]:
def build_model_cv(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X.values, y.values)
    
    pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                    ('tfidf', TfidfTransformer()),
                    ('multi', MultiOutputClassifier(RandomForestClassifier()))])
    
    parameters = {
        'multi__n_estimators': [50, 100, 200],
        'multi__min_samples_split': [2, 3, 4]}
    
    cv = GridSearchCV(pipeline, param_grid=parameters)
    
    cv.fit(X_train, y_train)
    
    y_pred = cv.predict(X_test)
    
    display_result(y_test, y_pred)


In [43]:
build_model_cv(X, y)

ValueError: Invalid parameter min_samples_split for estimator MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1). Check the list of available parameters with `estimator.get_params().keys()`.

### 6. Improve your model
Use grid search to find better parameters. 

In [19]:
parameters = {
        'clf__n_estimators': [50, 100, 200],
        'clf__min_samples_split': [2, 3, 4]}

cv = GridSearchCV(pipeline, param_grid=parameters)
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)
print("\nBest Parameters:", cv.best_params)

ValueError: Invalid parameter min_samples_split for estimator MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1). Check the list of available parameters with `estimator.get_params().keys()`.

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.